In [3]:
from pathlib import Path
import os
import sys
import pandas as pd
from unidecode import unidecode
import unicodedata
import numpy as np
import math as m
import re
from openpyxl import load_workbook
import math

In [4]:
# FUNCIONES IMPORTANTES
 # FUNCIONES IMPORTANTES
 #------------ Remueve acentos 
def remove_accents(input_str):
    # Normalizar el string a su forma combinada
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    # Filtrar y mantener solo los caracteres que no son diacríticos
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
# ----------- Crea data de zonas 
def Zone_data(Hoja,type):
    #excel = "diccionario_Benja/Diccionario_EMTP_DIgSILENT_BVega_v4.xlsx"
    excel = "diccionario_Benja/Zonas_DIgSILENT.xlsx"
    data= pd.read_excel(excel,sheet_name=Hoja)
    if type == "PV":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']  # Reemplaza con los nombres de las columnas que deseas
    elif type == "WP":
         columnas_deseadas = ['Name1','Zona DIgSILENT','Nombre DIgSILENT']     
    elif type == "SG":
        columnas_deseadas = ['Name1', 'Name2','Name3','Zona DIgSILENT','Nombre DIgSILENT']    
    elif type == "PMGD":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']  
    elif type == "CCSS":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']    
    elif type =="Cargas":
        columnas_deseadas = ['Carga EMTP','Zona DIgSILENT']   
        
    # Filtra el DataFrame para que contenga solo las columnas deseadas
    dataframe_filtrado = data[columnas_deseadas]
    #return dataframe_filtrado.dropna()   
    return dataframe_filtrado   

def Transformación_MW_MVAR(cadena):
    #cadena_sin_mas = cadena[1:]
    #final= float(cadena_sin_mas)/(10**6)
    final= float(cadena)/(10**6)
    final=round(final,2)
    return (final)
def obtener_V_op(cadena):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', cadena)
    indices_pares = [i for i in range(len(numeros)) if i % 2 == 0]
    valores_pares = [numeros[i] for i in indices_pares]
    lista_V = [round(float(elemento),2) for elemento in valores_pares]
    if lista_V[0] == lista_V[1] == lista_V[2]:
        return lista_V[0]
    else:
        return "desbanceado"
# toma un el string que tiene "V+anglo y me entrega solo el "V y redondeado" 
def separar_numeros(s):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', s)
    if numeros:
        numero_1 = float(numeros[0])  # Convertir a float
        return round(numero_1, 2)
# obtengo cual es la tension nominal a partir del resultado
def analizar_numero(n):
    if 90 <= n + 20 <= 130 or 90 <= n - 20 <= 130:
        return 110
    elif 200 <= n + 20 <= 240 or 200 <= n - 20 <= 240:
        return 220
    elif 46 <= n + 20 <= 86 or 46 <= n - 20 <= 86:
        return 66
    elif 10 <= n + 7 <= 23.2 or 7 <= n - 7 <= 23.2:
        return 13.2    
    elif 0.1 <= n + 2 <= 4 or 0.1 <= n -2 <= 4:
        return 0.6     
    else:
        return None  # Si no cae en ninguna categoría

def analizar_numero2(n):
    if n + 20 <= 130 and 90 <= n - 20:
        return 110
    elif  n + 20 <= 240 and 200 <= n - 20:
        return 220
    elif  n + 20 <= 86 and 46 <= n - 20:
        return 66
    elif  n + 1.799 <= 14.99 and 10.2 <= n -2 :
        return 13.2    
    elif 13.8 <= n - 1  or   math.trunc(n)==15:
        return 15  
    elif 0.1 <= n + 2 <= 4 or 0.1 <= n -2 <= 4:
        return 0.6     
    else:
        return None  # Si no cae en ninguna categoría


In [5]:
# Ruta de la carpeta en la que deseas buscar los archivos HTML
ruta_carpeta = '.'

# Listar todos los archivos con extensión .html en la carpeta
archivos_html = [archivo for archivo in os.listdir(ruta_carpeta) if archivo.endswith('.html')]
c=1
a = "Data"+str(c)
a

'Data1'

In [24]:
# Aqui coloco el nombre del archivo como string
archivo = "2024.11.14 1100 SEN 2030 Sur paso 1_lf.html"
#archivo = "2024.11.08 1047 SEN 2030 Norte_BVG Norte_lf.html"
#archivo = "SEN 2025 VRE Peak 85 VRE_g1_10 steps_lf.html"

excel = "Datos_nom_v1.xlsx"

# leo Excel y HTML
V_nominal = pd.read_excel(excel)
df= pd.read_html(archivo)
df=pd.DataFrame(df[0]) # Lo hago legible
#obtengo el nombre las columnas y las reescribo
fila_cero = df.iloc[0] 
df.columns =fila_cero.to_list()
#borro la fila 0 que tenia los nombres de las columnas
df = df.iloc[1:]

# reseteo los indices desde 0 y luego borro la primera columna indice
df=pd.DataFrame(df.reset_index())
#  obtengo solo las 5 primeras importantes 
# Central, Tipo, V ,P Q
lista_columnas = fila_cero.to_list()
lista_columnas= lista_columnas[:5] 
df = df[lista_columnas[:5]]

#--------------Data Generacion-------------------
valores_filtrar = ['PVbus', 'Slack', 'PQbus']
df_GEN = df[df['Type'].isin(valores_filtrar)]
# Data frame vacio de columnas definidas
columnas = ["V [kV]","P [MW]","Q [MVAr]"]
df_GEN_F = pd.DataFrame(columns=columnas)

df_GEN_F['P [MW]']=df_GEN['P (W)'].apply(Transformación_MW_MVAR)
df_GEN_F['Q [MVAr]']=df_GEN['Q (VAR)'].apply(Transformación_MW_MVAR)
df_GEN_F["V [kV]"] = df_GEN["Vabc (kVRMSLL,deg) phasor"].apply(obtener_V_op)
df_GEN_F['Vnom [kV]'] = df_GEN_F["V [kV]"].apply(analizar_numero) #Arreglo



#aqui la colummna device la expando para obtener 3 columnas de tal forma que coincida con lo anterior
Nombres = df_GEN['Device'].str.split('/', expand=True)
if  len(Nombres.columns) > 3 :
    Nombres.columns = ['Name1', 'Name2', 'NameLF',"Name4"]
    # Esto entrega la tabla de resultados de todas las centrales del HTML
    df_GEN_Final = pd.concat([Nombres,df_GEN_F], axis=1)
    df_GEN_Final = df_GEN_Final.drop(columns=['Name4']) #no tiene nada util
else : 
    Nombres.columns = ['Name1', 'Name2', 'NameLF']
    df_GEN_Final = pd.concat([Nombres,df_GEN_F], axis=1)





# PARCHES
df_GEN_Final.loc[df_GEN_Final['Name1'].str.contains('CCSS|CS', case=False, na=False), 'Vnom [kV]'] = 15
df_GEN_Final.loc[df_GEN_Final['Name1'].isin(['LF_CS_U15']), 'Vnom [kV]'] = 13.8
df_GEN_Final.loc[df_GEN_Final['Name1'].isin(['Central_IEM_CTM3'	]), 'Vnom [kV]'] = 20
df_GEN_Final.loc[df_GEN_Final['Name1'].isin(['Central_EL_Paso'	]), 'Vnom [kV]'] = 10.5

 ##------------------------Sobre escribo datos nominales------------------------------------
    
N = len(df_GEN_Final)
for fila in range(N):
    Name1= df_GEN_Final.at[fila,"Name1"]
    Name2= df_GEN_Final.at[fila,"Name2"]
    NameLF= df_GEN_Final.at[fila,"NameLF"]
    result = V_nominal[(V_nominal['Name1'] == Name1) &
                                 (V_nominal['Name2'] == Name2) & 
                                 (V_nominal['NameLF'] == NameLF)]
    if len(result)== 1:
        df_GEN_Final.at[fila,"Vnom [kV]"] =result["Tensión Nominal [kV]"]
    else: 
        continue 
# Escribo V pu 
df_GEN_Final['Tensión [pu]'] = df_GEN_Final["V [kV]"]/df_GEN_Final['Vnom [kV]']
pd.set_option('display.max_rows', 140)
#with pd.ExcelWriter('Resultados_.xlsx') as writer:
 #   df_GEN_F.to_excel(writer, sheet_name='LFcargas',index=False)
Tabla_LF_GEN=df_GEN_Final
Tabla_LF_GEN

C:\Users\fabian.medina\AppData\Local\Temp\ipykernel_16868\2010492910.py:72: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_GEN_Final.at[fila,"Vnom [kV]"] =result["Tensión Nominal [kV]"]


,Name1,Name2,NameLF,V [kV],P [MW],Q [MVAr],Vnom [kV],Tensión [pu]
0,Central_Ralco,Ralco_U1,LF_RALC_U1,14.08,125.35,23.61,13.8,1.020290
1,C_ANGOSTURA,Angostura_U1,LF2,13.80,74.00,-6.33,13.8,1.000000
2,Central_Canutillar,Canutillar_U1,LF_CANU_U1,12.47,25.00,10.00,13.8,0.903623
3,Central_Canutillar,Canutillar_U2,LF_CANU_U2,12.47,25.00,10.00,13.8,0.903623
4,Central_Colbun,Colbun_U1,LF_COLB_U1,14.20,100.10,15.51,13.8,1.028986
5,Central_Colbun,Colbun_U2,LF_COLB_U2,14.20,100.10,15.45,13.8,1.028986
6,Central_LaConfluencia,LaConfluencia_U1,LF_LCONFLUE_U1,15.10,28.00,-4.38,15.0,1.006667
7,Central_LaConfluencia,LaConfluencia_U2,LF_LCONFLUE_U2,15.20,28.00,-19.47,15.0,1.013333
8,Central_LaHiguera,LaHiguera_U1,LF_LHIGUER_U1,14.30,40.00,-1.18,13.8,1.036232
9,Central_LaHiguera,LaHuiguera_U2,LF_LHIGUER_U2,14.30,40.00,-1.18,13.8,1.036232


In [13]:
len(Nombres.columns)

3

# Resultados: LOAD FLOW (PV,WP,SG)

In [15]:
# Aqui coloco el nombre del archivo como string
#archivo =  "SEN 2025 VRE Peak 04072024 DBC 85 GFM New - H1_v4_lf.html"
#archivo = "SEN 2025 VRE Peak 85 VRE_g1_10 steps_lf.html"
archivo = "2024.11.08 1022 SEN 2030 Sur_BVG_lf"
#archivo = "2024.11.08 1047 SEN 2030 Norte_BVG Norte_lf.html"
excel = "Datos_nom_v1.xlsx"

def Resultados_Load_Flow(archivo_html,excel):

    Datos_Nominal_extras = pd.read_excel(excel)
    df= pd.read_html(archivo_html)
    columnas = ['Tensión en Bornes [kV]','Potencia Activa [MW]', 'Potencia Reactiva [Mvar]']
    columnas2 = ['Name1','Name2', 'EMTP Load Flow Component']
    df2 = pd.DataFrame(columns=columnas2)
    
    # Crear el DataFrame vacío con las columnas definidas
    df_FINAL = pd.DataFrame(columns=columnas)
    df=pd.DataFrame(df[0])
    # Se procede a transformar el dataframe de html para que este legible 

    #obtengo el nombre las columnas y las reescribo
    fila_cero = df.iloc[0] 
    df.columns =fila_cero.to_list()
    #borro la fila 0 que tenia los nombres de las columnas
    df = df.iloc[1:]
    # reseteo los indices desde 0 y luego borro la primera columna indice
    df=pd.DataFrame(df.reset_index())
    lista_columnas = fila_cero.to_list()
    #  obtengo solo las 5 primeras importantes
    lista_columnas= lista_columnas[:5] 
    df2 = df[lista_columnas[:5]]
    # Crear una lista que solo contenga las centrales y no las cargas 
 
    valores_filtrar = ['PVbus', 'Slack', 'PQbus']
    df_filtrado = df2[df2['Type'].isin(valores_filtrar)]

    df_FINAL['Potencia Activa [MW]']=df_filtrado['P (W)'].apply(Transformación_MW_MVAR)
    df_FINAL['Potencia Reactiva [Mvar]']=df_filtrado['Q (VAR)'].apply(Transformación_MW_MVAR)
    df_FINAL["Tensión en Bornes [kV]"] = df_filtrado["Vabc (kVRMSLL,deg) phasor"].apply(obtener_V_op)
    df_FINAL['Tensión Nominal [kV]'] = df_FINAL["Tensión en Bornes [kV]"].apply(analizar_numero)
   

    #aqui la colummna device la expando para obtener 3 columnas de tal forma que coincida con lo anterior
    nuevo_df = df_filtrado['Device'].str.split('/', expand=True)
    nuevo_df.columns = ['Name1', 'Name2', 'NameLF',"Name4"]

    # Esto entrega la tabla de resultados de todas las centrales del HTML
    df_Resultados = pd.concat([nuevo_df, df_FINAL], axis=1)
    pd.set_option('display.max_rows', 140)

    #---------------------------- saco los acentos 
    #df_Resultados['Name1'] = df_Resultados['Name1'].apply(lambda x: unidecode(x))
   # df_Resultados['Name2'] = df_Resultados['Name2'].apply(lambda x: unidecode(x))
    df_Resultados = df_Resultados.drop(columns=['Name4']) #no tiene nada util

    ##------------------------Sobre escribo datos nominales------------------------------------
    
    N = len(df_Resultados)
    for fila in range(N):
        Name1= df_Resultados.at[fila,"Name1"]
        Name2= df_Resultados.at[fila,"Name2"]
        NameLF= df_Resultados.at[fila,"NameLF"]
        result = Datos_Nominal_extras[(Datos_Nominal_extras['Name1'] == Name1) &
                                 (Datos_Nominal_extras['Name2'] == Name2) & 
                                 (Datos_Nominal_extras['NameLF'] == NameLF)]
        if len(result)== 1:
            df_Resultados.at[fila,"Tensión Nominal [kV]"] =result["Tensión Nominal [kV]"]
        else: 
            continue 
    

    df_Resultados['Tensión [pu]'] = df_Resultados["Tensión en Bornes [kV]"]/df_Resultados['Tensión Nominal [kV]']

    return(df_Resultados)

Tabla_LF_GEN=Resultados_Load_Flow(archivo,excel)
with pd.ExcelWriter('Resultados_.xlsx') as writer:
    Tabla_LF_GEN.to_excel(writer, sheet_name='GEN',index=False)

Tabla_LF_GEN

,Name1,Name2,NameLF,Tensión en Bornes [kV],Potencia Activa [MW],Potencia Reactiva [Mvar],Tensión Nominal [kV],Tensión [pu]
0,Central_IEM_CTM3,IEM,LF_IEM,20.80,248.62,-1.40,13.2,1.575758
1,SVC_DOMEYKO,PVinit,LF,0.58,0.00,9.04,13.2,0.043939
2,STAT_LAGUNAS,PVinit,LF,0.59,0.00,5.90,13.2,0.044697
3,PFV_Pampa_Tigre_New,PQinit,LF,227.98,77.71,-0.00,220.0,1.036273
4,PFV_Willka2_New,PQinit,LF,230.57,61.33,-27.75,220.0,1.048045
5,PFV_Palm_Huayca_New,PQinit,LF,228.41,29.20,-0.00,220.0,1.038227
6,PFV_Tirana_Oeste_OG,PQinit,LF,228.44,69.33,-0.00,220.0,1.038364
7,PFV_Solar_Toro_OG,PQinit,LF,224.97,41.61,-0.00,220.0,1.022591
8,PFV_Socompa_OG,PQinit,LF,223.92,144.43,-0.00,220.0,1.017818
9,PMGD_PFV_Calama220,PQinit,LF,223.68,41.85,0.00,220.0,1.016727


# Resultados: LOAD FLOW Cargas

In [29]:
#archivo = "SEN 2025 VRE Peak 85 VRE_g1_10 steps_lf.html"
archivo = "2024.11.08 1047 SEN 2030 Norte_BVG Norte_lf.html"
archivo = "2024.11.14 1100 SEN 2030 Sur paso 1_lf.html"
def Resultados_Load_Flow_cargas(archivo_html):


    df= pd.read_html(archivo_html)
    columnas = ["V [kV]","P [MW]","Q [MVAr]"]
    #columnas = ['Tensión en Bornes [kV]','Potencia Activa [MW]', 'Potencia Reactiva [Mvar]']
    columnas2 = ['Name1','Name2', 'EMTP Load Flow Component']
    df2 = pd.DataFrame(columns=columnas2)
    
    # Crear el DataFrame vacío con las columnas definidas
    df_FINAL = pd.DataFrame(columns=columnas)
    df=pd.DataFrame(df[0])
    # Se procede a transformar el dataframe de html para que este legible 

    #obtengo el nombre las columnas y las reescribo
    fila_cero = df.iloc[0] 
    df.columns =fila_cero.to_list()
    #borro la fila 0 que tenia los nombres de las columnas
    df = df.iloc[1:]
    # reseteo los indices desde 0 y luego borro la primera columna indice
    df=pd.DataFrame(df.reset_index())
    lista_columnas = fila_cero.to_list()
    #  obtengo solo las 5 primeras importantes
    lista_columnas= lista_columnas[:5] 
    df2 = df[lista_columnas[:5]]
    # Crear una lista que solo contenga las centrales y no las cargas 


    valores_filtrar2 = ['PQload']
    df_filtrado2 = df2[df2['Type'].isin(valores_filtrar2)]
 
    #valores_filtrar = ['PVbus', 'Slack', 'PQbus']
    #df_filtrado = df2[df2['Type'].isin(valores_filtrar)]

    df_FINAL["P [MW]"]=df_filtrado2['P (W)'].apply(Transformación_MW_MVAR)
    df_FINAL["Q [MVAr]"]=df_filtrado2['Q (VAR)'].apply(Transformación_MW_MVAR)
    df_FINAL["V [kV]"] = df_filtrado2["Vabc (kVRMSLL,deg) phasor"].apply(separar_numeros)
    df_FINAL['Vnom [kV]'] = df_FINAL["V [kV]"].apply(analizar_numero)
    df_FINAL['V [pu]'] = df_FINAL["V [kV]"]/df_FINAL['Vnom [kV]']
  

    nuevo_df2 =df_filtrado2["Device"] 
    # Esto entrega la tabla de resultados de todas las centrales del HTML
    
    df_Resultados2 = pd.concat([nuevo_df2, df_FINAL], axis=1)



    #---------------------------- saco los acentos 
    #df_Resultados['Name1'] = df_Resultados['Name1'].apply(lambda x: unidecode(x))
    #df_Resultados['Name2'] = df_Resultados['Name2'].apply(lambda x: unidecode(x))


    ##------------------------------------------------------------------------------------------
   
    # que me entregue las columnas de 3 en 3
    df_Resultados2 = df_Resultados2.iloc[::3]
    #reseteo los indices 
    df_Resultados2 = df_Resultados2.reset_index(drop=True)
    #borro el nombre "Load_a del string de su"
    df_Resultados2['Device'] = df_Resultados2['Device'].str.replace('/Load_a', '', regex=False)
    return(df_Resultados2)

Tabla_LF_cargas= pd.DataFrame(Resultados_Load_Flow_cargas(archivo))
pd.set_option('display.max_rows', 200)
Tabla_LF_cargas

,Device,V [kV],P [MW],Q [MVAr],Vnom [kV],V [pu]
0,ATR_LASTARR_220,206.13,10.27,-0.35,220,0.936955
1,ATR_METR_220_1,212.84,18.48,-1.30,220,0.967455
2,ATR_PICHIRRO,198.89,16.91,2.73,220,0.904045
3,ATR_VALDIV_220,200.24,15.07,2.47,220,0.910182
4,CHARRUA_154,229.51,59.44,20.82,220,1.043227
5,CLAJA_220,227.61,4.07,-2.69,220,1.034591
6,CONCEPCION_220,220.34,49.23,11.80,220,1.001545
7,FCR_154,226.51,34.70,15.27,220,1.029591
8,INFOR_220,225.12,6.72,2.34,220,1.023273
9,L_LLANQUI,196.75,12.27,2.60,220,0.894318


# Resultados: LOAD FLOW Cargas

In [ ]:
#Resultados_Load_Flow2(archivo)
#a= pd.DataFrame(Resultados_Load_Flow_cargas(archivo))
with pd.ExcelWriter('Resultados_HTML.xlsx') as writer:

    Tabla_LF_cargas.to_excel(writer, sheet_name='LFcargas',index=False)
    Tabla_LF_GEN.to_excel(writer, sheet_name='LFGEN',index=False)
